**KASIFIKASI OUTPUT TARGET**

0.   BELOK KANAN TAJAM
1.   BELOK KIRI TAJAM
2.   BELOK KANAN SIKU
3.   BELOK KIRI SIKU



# Mount to Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import joblib
from tensorflow import keras

model_filename = 'model_mlp.h5'
drive_path_model = '/content/drive/MyDrive/Colab Notebooks/MLP-Learning-LineFollower/models/'
model_full_path = drive_path_model + model_filename

try:
    model = keras.models.load_model(model_full_path)
    print(f"✅ Model '{model_filename}' berhasil dimuat dari Drive.")
except Exception as e:
    print(f"❌ Error saat memuat model dari Drive: {e}")

# menggunakan StandardScaler dan menyimpannya pada 'scaler.pkl'
scaler_filename = 'scaler.pkl'
scaler_full_path = drive_path_model + scaler_filename

try:
    scaler = joblib.load(scaler_full_path)
    print(f"✅ Scaler '{scaler_filename}' berhasil dimuat dari Drive.")
except FileNotFoundError:
    print(f"❌ Error: File scaler.pkl tidak ditemukan di {scaler_full_path}. Pastikan path dan nama file sudah benar!")


# Testing Model

In [ ]:
import pandas as pd
import numpy as np

JUMLAH_FITUR = 10
JUMLAH_KELAS = 4
FITUR_NAMES = [
    'IR 1_log', 'IR 2_log', 'IR 3_log', 'IR 4_log', 'IR 5_log',
    'IR 6_log', 'IR 7_log', 'IR 8_log', 'IR 9_log', 'IR 10_log'
]
class_labels = [f"Kelas {i}" for i in range(JUMLAH_KELAS)]

print(f"Masukkan {JUMLAH_FITUR} nilai fitur mentah secara terpisah:")

feature_values_list = []

try:
    for i in range(JUMLAH_FITUR):
        prompt_name = FITUR_NAMES[i].replace('_log', '')
        input_value = input(f"Input {prompt_name} (Fitur ke-{i + 1}): ")
        feature_values_list.append(float(input_value))

    raw_input = np.array(feature_values_list, dtype=np.float32)

    print(f"\n✅ Input mentah berhasil diolah: {raw_input}")

    sample_to_transform = raw_input.reshape(1, -1)
    log_transformed_input = np.log1p(sample_to_transform)
    input_df = pd.DataFrame(log_transformed_input, columns=FITUR_NAMES)
    normalized_input = scaler.transform(input_df)

    prediction_probs = model.predict(normalized_input)

    predicted_class_index = np.argmax(prediction_probs, axis=1)[0]
    confidence = prediction_probs[0][predicted_class_index]

    print("\n--- Hasil Prediksi Model ---")
    print(f"Input Dinormalisasi (Siap Model) : \n{normalized_input.round(4)}")
    print(f"Probabilitas Prediksi (Raw Output) : {prediction_probs.round(4)}")

    print(f"\nHasil Klasifikasi Akhir:")
    print(f"Kelas yang Diprediksi (Indeks): {predicted_class_index}")
    print(f"Label Prediksi: {class_labels[predicted_class_index]}")
    print(f"Tingkat Kepercayaan: {confidence*100:.2f}%")

    print("\nDetail Probabilitas:")
    for i, prob in enumerate(prediction_probs[0]):
         print(f"- {class_labels[i]}: {prob*100:.2f}%")

except ValueError:
    print("\n❌ ERROR: Pastikan semua input yang Anda masukkan adalah angka (integer atau float) yang valid.")
except Exception as e:
    print(f"\n❌ Terjadi kesalahan tak terduga selama prediksi: {e}")